<a href="https://colab.research.google.com/github/abeebyekeen/DLforBeginners/blob/main/day1_session3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#day 1 session 3: visualization of loss surface and optimization

Downhill Simplex Optimization (or Nelder–Mead method [wiki](https://en.wikipedia.org/wiki/Nelder%E2%80%93Mead_method) [scipy](https://docs.scipy.org/doc/scipy/reference/optimize.minimize-neldermead.html)) in 2D parameter space




**tasks**:
1. implement the perceptron model and BCE loss function
2. run the pipeline and observe how Downhill Simplex Optimization works
3. to view its behavior near the minimum more clearly with higher contrast, plot only the area around the minimum
4. use Downhill Simplex Optimization on task 2 from session 2
5. use Downhill Simplex Optimization on task 3 from session 2
6. use Downhill Simplex Optimization on task 4 from session 2
7. apply Downhill Simplex Optimization on multiclass classification problem (create your own dataset, perceptron, loss)

##part 0: dependencies and functions
utility code that you don't have to read

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from IPython.display import Javascript

In [ ]:
# function for creating a binary classification dataset
def create_data_1(N):
    X0 = np.random.rand(N,2).astype(np.float32)-0.5
    X1 = np.random.rand(N,2).astype(np.float32)-0.5
    X0[:,0] = X0[:,0] - 0.25
    X1[:,0] = X1[:,0] + 0.25
    X0[:,1] = X0[:,1] - 0.25
    X1[:,1] = X1[:,1] + 0.25
    X = np.concatenate((X0,X1), axis=0)
    Y0 = np.zeros((N)).astype(np.float32)
    Y1 = np.ones( (N)).astype(np.float32)
    Y = np.concatenate((Y0,Y1), axis=0)
    # shuffle X and Y
    new_id = np.random.permutation(2*N)
    X = X[new_id]
    Y = Y[new_id]
    return X,Y

In [ ]:
# function for creating a binary classification dataset
def create_data_2(N):
    r0 = np.random.rand(N).astype(np.float32)+1.0
    r1 = np.random.rand(N).astype(np.float32)+3.0
    theta0 = (np.random.rand(N)-0.5).astype(np.float32) * 2.0 * np.pi
    theta1 = (np.random.rand(N)-0.5).astype(np.float32) * 2.0 * np.pi
    X0 = np.column_stack((r0*np.cos(theta0),r0*np.sin(theta0)))
    X1 = np.column_stack((r1*np.cos(theta1),r1*np.sin(theta1)))
    X = np.concatenate((X0,X1), axis=0)
    Y0 = np.zeros((N)).astype(np.float32)
    Y1 = np.ones( (N)).astype(np.float32)
    Y = np.concatenate((Y0,Y1))
    # shuffle X and Y
    new_id = np.random.permutation(2*N)
    X = X[new_id]
    Y = Y[new_id]
    return X,Y

In [ ]:
# function for creating a binary classification dataset
def create_data_3(N):
    X1 = np.random.rand(N,2).astype(np.float32)
    X2 = np.random.rand(N,2).astype(np.float32)
    X3 = np.random.rand(N,2).astype(np.float32)
    X4 = np.random.rand(N,2).astype(np.float32)
    X2[:,0] = X2[:,0] - 1.0
    X3 = X3 - 1.0
    X4[:,1] = X4[:,1] - 1.0
    X = np.concatenate((X1,X2,X3,X4), axis=0)
    Y1 = np.zeros((N)).astype(np.float32)
    Y2 = np.ones( (N)).astype(np.float32)
    Y3 = np.zeros((N)).astype(np.float32)
    Y4 = np.ones( (N)).astype(np.float32)
    Y = np.concatenate((Y1,Y2,Y3,Y4), axis=0)
    # shuffle X and Y
    new_id = np.random.permutation(4*N)
    X = X[new_id]
    Y = Y[new_id]
    return X,Y

In [ ]:
# function for visualizing dataset
def visualize_data(X,Y,figuretitle):
    fig = plt.figure()
    ax = plt.gca()
    C0 = ax.scatter(X[Y==0,0],X[Y==0,1],c='b')
    C1 = ax.scatter(X[Y==1,0],X[Y==1,1],c='r')
    #c = ['b' if Y[i]==0 else 'r' for i in range(len(Y))]
    #scatter = ax.scatter(X[:,0], X[:,1], c=c)
    ax.set_aspect('equal')
    ax.xaxis.set_major_locator(ticker.MultipleLocator(base=0.2))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.2))
    legend1 = ax.legend((C0,C1),('y=0','y=1'),loc='upper left',)
    plt.xlabel('x0')
    plt.ylabel('x1')
    plt.title(figuretitle)
    plt.plot()

In [ ]:
# function for visualizing loss function
def visualize_loss(Jout, w0_start, w0_end, w1_start, w1_end, step):
    x = np.arange(w0_start - 0.5*step, w0_end + 1.0*step, step)
    y = np.arange(w1_start - 0.5*step, w1_end + 1.0*step, step)
    z = Jout
    fig, ax = plt.subplots()
    c = ax.pcolormesh(x, y, z)
    ax.set_aspect('equal')
    ax.set_xticks(x[:-1]+0.5*step)
    ax.set_yticks(y[:-1]+0.5*step)
    fig.colorbar(c, ax=ax)

In [ ]:
#modified from a piece of code from erwan-simon: "https://gist.github.com/erwan-simon/e3baef06a00bb9a39a6968acf78121ee"
#from torch.autograd import Variable
def plot_decision_boundary(X,Y,w,model):
    dataset= X
    labels = Y
    color_map='Paired'
    color_map = plt.get_cmap(color_map)
    # Define region of interest by data limits
    x0min, x0max = dataset[:, 0].min() - 1, dataset[:, 0].max() + 1
    x1min, x1max = dataset[:, 1].min() - 1, dataset[:, 1].max() + 1
    steps = 1000
    x0_span = np.linspace(x0min, x0max, steps)
    x1_span = np.linspace(x1min, x1max, steps)
    x0s, x1s = np.meshgrid(x0_span, x1_span)

    # Make predictions across region of interest
    #model.eval()
    #labels_predicted = model(Variable(torch.from_numpy(np.c_[xx.ravel(), yy.ravel()]).float()))
    labels_predicted = model(w, np.c_[x0s.ravel(), x1s.ravel()])
    # Plot decision boundary in region of interest
    #labels_predicted = [0 if value <= 0.5 else 1 for value in labels_predicted.detach().numpy()]
    z = np.array(labels_predicted).reshape(x0s.shape)

    fig, ax = plt.subplots()
    contourf_ = ax.contourf(x0s, x1s, z, cmap=color_map, alpha=0.5)
    cbar = fig.colorbar(contourf_)
    #cbar.set_clim( vmin, vmax )

    # Get predicted labels on training data and plot
    #train_labels_predicted = model(dataset)

    C0 = ax.scatter(X[Y==0,0],X[Y==0,1],c='b')
    C1 = ax.scatter(X[Y==1,0],X[Y==1,1],c='r')
    ax.set_aspect('equal')
    #ax.set(xlim=(-3, 3), ylim=(-3, 3))
    plt.xlabel('x0')
    plt.ylabel('x1')
    plt.title("infering data, true labels, decision bounary")
    plt.plot()

In [ ]:
# print the rates of TN, FP, FN, TP, and ACC
def evaluate(Y,Y_pred):
    total = len(Y)
    TN = np.count_nonzero(np.logical_and(Y==0.0,Y_pred==0.0)) / total
    FP = np.count_nonzero(np.logical_and(Y==0.0,Y_pred==1.0)) / total
    FN = np.count_nonzero(np.logical_and(Y==1.0,Y_pred==0.0)) / total
    TP = np.count_nonzero(np.logical_and(Y==1.0,Y_pred==1.0)) / total
    ACC = TN+TP
    print('TN rate:',TN,', FP rate: ',FP,', FN rate: ',FN,', TP rate: ',TP)
    print('ACC=',ACC)

In [ ]:
# modified from
# https://github.com/fchollet/nelder-mead/blob/master/nelder_mead.py
import copy

'''
    Pure Python/Numpy implementation of the Nelder-Mead algorithm.
    Reference: https://en.wikipedia.org/wiki/Nelder%E2%80%93Mead_method
'''


def nelder_mead(f, x_start,args,
                step=0.1, no_improve_thr=10e-6,
                no_improv_break=10, max_iter=0,
                alpha=1., gamma=2., rho=-0.5, sigma=0.5):
    '''
        @param f (function): function to optimize, must return a scalar score
            and operate over a numpy array of the same dimensions as x_start
        @param x_start (numpy array): initial position
        @param step (float): look-around radius in initial step
        @no_improv_thr,  no_improv_break (float, int): break after no_improv_break iterations with
            an improvement lower than no_improv_thr
        @max_iter (int): always break after this number of iterations.
            Set it to 0 to loop indefinitely.
        @alpha, gamma, rho, sigma (floats): parameters of the algorithm
            (see Wikipedia page for reference)

        return: tuple (best parameter array, best score)
    '''

    # init
    dim = len(x_start)
    prev_best = f(x_start,*args)
    no_improv = 0
    res = [[x_start, prev_best]]

    for i in range(dim):
        x = copy.copy(x_start)
        x[i] = x[i] + step
        score = f(x,*args)
        res.append([x, score])

    record = []
    record.append(copy.deepcopy(res))
    # simplex iter
    iters = 0
    while 1:
        print()
        # order
        res.sort(key=lambda x: x[1])
        best = res[0][1]

        # break after max_iter
        if max_iter and iters >= max_iter:
            return res[0], record
        iters += 1
        #print(iters)

        # break after no_improv_break iterations with no improvement
        print('...best so far:', best)

        if best < prev_best - no_improve_thr:
            no_improv = 0
            prev_best = best
        else:
            no_improv += 1

        if no_improv >= no_improv_break:
            return res[0], record

        # centroid
        x0 = [0.] * dim
        for tup in res[:-1]:
            for i, c in enumerate(tup[0]):
                x0[i] += c / (len(res)-1)

        # reflection
        xr = x0 + alpha*(x0 - res[-1][0])
        rscore = f(xr,*args)
        if res[0][1] <= rscore < res[-2][1]:
            del res[-1]
            res.append([xr, rscore])
            record.append(copy.deepcopy(res))
            continue

        # expansion
        if rscore < res[0][1]:
            xe = x0 + gamma*(x0 - res[-1][0])
            escore = f(xe,*args)
            if escore < rscore:
                del res[-1]
                res.append([xe, escore])
                record.append(copy.deepcopy(res))
                continue
            else:
                del res[-1]
                res.append([xr, rscore])
                record.append(copy.deepcopy(res))
                continue

        # contraction
        xc = x0 + rho*(x0 - res[-1][0])
        cscore = f(xc,*args)
        if cscore < res[-1][1]:
            del res[-1]
            res.append([xc, cscore])
            record.append(copy.deepcopy(res))
            continue

        # reduction
        x1 = res[0][0]
        nres = []
        for tup in res:
            redx = x1 + sigma*(tup[0] - x1)
            score = f(redx,*args)
            nres.append([redx, score])
        res = nres
        record.append(copy.deepcopy(res))


#if __name__ == "__main__":
#    # test
#    import math
#    import numpy as np
#
#    def f(x):
#        return math.sin(x[0]) * math.cos(x[1]) * (1. / (abs(x[2]) + 1))
#
#    print nelder_mead(f, np.array([0., 0., 0.]))

##part 1: preparations

In [ ]:
# create dataset
SEED = 2023
#random.seed(SEED)
np.random.seed(SEED)
N=1000
X1,Y1=create_data_1(N)
X1_train = X1[:N]
Y1_train = Y1[:N]
X1_infer = X1[N:]
Y1_infer = Y1[N:]

display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 9999})'''))
visualize_data(X1,Y1,"dataset")
visualize_data(X1_train,Y1_train,"dataset train")
visualize_data(X1_infer,Y1_infer,"dataset infer")

In [ ]:
# implement perceptron
def MyPerceptron1(w,x):
    y = None # edit here
    return y

# implement loss function
def MyLossFunction(w,x,y,model):
    return 0 # edit here

##part 2: train

In [ ]:
# run Downhill Simplex Optimization (Nelder–Mead method)
res,record = nelder_mead(MyLossFunction, x_start = np.array([0.0,0.0]), args=(X1_train,Y1_train,MyPerceptron1) )

In [ ]:
# in order to plot the optimization process with animation
# save the values of weights at each iteration (together with losses)
n_steps = len(record)
record_w = np.zeros([n_steps,3,2],dtype=np.float32)
record_l = np.zeros([n_steps,3],dtype=np.float32)
for i in range(n_steps):
    for j in range(3):
        record_w[i,j,:] = record[i][j][0]
        record_l[i,j]   = record[i][j][1]

In [ ]:
# print weights at each iteration
print(record_w)

In [ ]:
# print losses at each iteration
print(record_l)

In [ ]:
# calculate loss values at each point in parameter space
# https://matplotlib.org/stable/gallery/mplot3d/surface3d.html
w0s = np.arange(-1.0, 7.0, 0.1) # modify this to see the minimum area with higher contrast during task 3
w1s = np.arange(-1.0, 7.0, 0.1) # modify this to see the minimum area with higher contrast during task 3
w0s, w1s = np.meshgrid(w0s, w1s)
L = np.zeros_like(w0s)
for i in range(len(w0s)):
    for j in range(len(w1s)):
        W = np.array([w0s[i,j],w1s[i,j]])
        L[i,j] = MyLossFunction(W,X1_train,Y1_train,MyPerceptron1)

In [ ]:
# visualize the optimization process
from matplotlib import rc
rc('animation', html='jshtml')
import IPython
import matplotlib.animation as animation
from matplotlib import cm

fig, ax = plt.subplots()
#surf = ax.plot_surface(w0s, w1s, L, cmap=cm.coolwarm,linewidth=0, antialiased=False)
surf = ax.pcolormesh(w0s, w1s, L, cmap=cm.coolwarm,linewidth=0, antialiased=False)

line1 = ax.plot([record_w[0,0,0], record_w[0,1,0]], [record_w[0,0,1], record_w[0,1,1]], color='k', linestyle='-', linewidth=1)[0]
line2 = ax.plot([record_w[0,1,0], record_w[0,2,0]], [record_w[0,1,1], record_w[0,2,1]], color='k', linestyle='-', linewidth=1)[0]
line3 = ax.plot([record_w[0,2,0], record_w[0,0,0]], [record_w[0,2,1], record_w[0,0,1]], color='k', linestyle='-', linewidth=1)[0]

def update(frame):
    line1.set_xdata([record_w[frame,0,0], record_w[frame,1,0]])
    line1.set_ydata([record_w[frame,0,1], record_w[frame,1,1]])
    line2.set_xdata([record_w[frame,1,0], record_w[frame,2,0]])
    line2.set_ydata([record_w[frame,1,1], record_w[frame,2,1]])
    line3.set_xdata([record_w[frame,2,0], record_w[frame,0,0]])
    line3.set_ydata([record_w[frame,2,1], record_w[frame,0,1]])
    return (line1, line2, line3)

fig.colorbar(surf, shrink=0.5, aspect=5)
plt.xlabel('w0')
plt.ylabel('w1')
plt.title("loss")
plt.show()

ani = animation.FuncAnimation(fig=fig, func=update, frames=n_steps, interval=30)
ani


## part 3: infer

In [ ]:
Y1_infer_pred = MyPerceptron1(res[0],X1_infer)
Y1_infer_pred[Y1_infer_pred<=0.5] = 0.0
Y1_infer_pred[Y1_infer_pred> 0.5] = 1.0

In [ ]:
evaluate(Y1_infer,Y1_infer_pred)

In [ ]:
plot_decision_boundary(X1_infer,Y1_infer_pred,res[0],MyPerceptron1)